In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import numpy as np
import math
import copy
import torch
import torch.nn as nn
import torch.nn.functional as f

from game import *
from player import *

In [34]:
PATH = 'C:/Users/abhis/Downloads/New folder/project/Snake AI'

In [35]:
def generate_individual_brain(input_size, num_hidden_layers, hidden_layers_size, output_size):
    layers = []
    layers.append(nn.LayerNorm(input_size))
    layers.append(nn.Linear(input_size, hidden_layers_size[0]))
    for layer in range(num_hidden_layers-1):
        layers.append(nn.Linear(hidden_layers_size[layer], hidden_layers_size[layer+1]))
        # layers.append(nn.Tanh())
    layers.append(nn.Linear(hidden_layers_size[-1], output_size))
    # layers.append(nn.Tanh())
    layers.append(nn.Softmax())
    AI_bc = nn.Sequential(*layers, )
    for layer in AI_bc.parameters():
        layer.requires_grad = False
    return AI_bc

In [36]:
# def generate_brains(pop_size, input_size, num_hidden_layers, hidden_layers_size, output_size):
#     brains = []
#     for brain in range(pop_size):
#         # AI_bc
#         AI_bc = generate_individual_brain(input_size, num_hidden_layers, hidden_layers_size, output_size)
#         brains.append(AI_bc)
#     return brains

In [37]:
def get_move(input, i):
    dir = brains[i](input).argmax()
    return DIRECTIONS[dir]

In [38]:
def generate_input(game, window_size):
    
    head = game.snake_heads[0]
    positions = game.get_all_eligible_neighbors(head, borders=True, window_size=window_size)
    input_vector = []
    for i, position in enumerate(positions):
        # import pdb; pdb.set_trace();
        input_vector_val = game.extract_position(position)
        if input_vector_val == game.food_val:
            input_vector_val = 1
        elif input_vector_val == game.body_val:
            input_vector_val = -1
        elif input_vector_val == game.default_board_val:
            input_vector_val = 0
        elif input_vector_val == game.out_of_board:
            input_vector_val = -1
        input_vector.append(input_vector_val)
    input_vector.append(head[0])
    input_vector.append(head[1])
    input_vector.append(game.food_positions[0][0])
    input_vector.append(game.food_positions[0][1])
    input_vector.append(head[0]-game.food_positions[0][0])
    input_vector.append(head[1]-game.food_positions[0][1])
    input_vector = torch.tensor(input_vector).float().reshape(-1, input_size)
    return input_vector

In [39]:
def run_one_trial(board_size, set_seed, i, window_size):
    moves = 0
    game = Game(board_size, set_seed=set_seed)
    # print(game.board)
    # game.print_board(True)
    while game.zinda_hai_ki_nahi and moves <= move_limit:
        input = generate_input(game, window_size)
        dir = get_move(input, i)
        game.move(dir)
        moves += 1
    return moves, len(game.snakes[0])

In [40]:
def one_gen(board_size, set_seed, gen=0, window_size=3):
    max_moves = [0] * pop_size
    max_scores = [0] * pop_size
    avg_moves = [[]] * pop_size
    avg_scores = [[]] * pop_size
    for i in range(pop_size):    
        individual_moves = []
        individual_scores = []
        for j in range(num_trials):
            # print(f'trial {j} for pop {i} for gen {gen}')
            moves, score = run_one_trial(board_size, set_seed, i, window_size)
            avg_moves[i].append(moves)
            avg_scores[i].append(score)
            individual_moves.append(moves)
            individual_scores.append(score)
            if moves > max_moves[i]:
                max_moves[i] = moves
            if score > max_scores[i]:
                max_scores[i] = score
        avg_moves[i] = sum(avg_moves[i])/len(avg_moves[i])
        avg_scores[i] = sum(avg_scores[i])/len(avg_scores[i])
        # print(f'ran {j} trials for individual {i} out of {pop_size} for gen {gen}, max move - {max(individual_moves)}, max score - {max(individual_scores)}')
    # print(f'max moves for gen {gen} - {max_moves}')
    # print(f'max scores  for gen {gen} - {max_scores}')
    return np.array(max_moves), np.array(max_scores), np.array(avg_moves), np.array(avg_scores)

In [41]:
def save_brains(PATH):
    for i, brain in enumerate(brains):
        torch.save(brain, f'{PATH}_brain_{i}.pt')

def load_brains(PATH):
    for pop in range(pop_size):
        brains[pop] = torch.load(f'{PATH}_brain_{pop}.pt')
        brains[pop].eval()

In [90]:
def mutate(brain):
    print(brain[0][1].weight)
    for layer in brain:
        if isinstance(layer, nn.Linear):
            # print('original')
            # print(layer.weight[0])
            # print('new')
            # new_weights = layer.weight + ((torch.rand(layer.weight.shape)<self.mutation_rate).long() * torch.normal(layer.weight.shape) * self.mutation_change)
            new_weights = layer.weight + ((torch.rand(layer.weight.shape)<mutation_rate).long() * torch.empty(layer.weight.shape).normal_(mean=0,std=1) * mutation_change)
            layer.weight = nn.Parameter(new_weights, requires_grad=False)
            # print(layer.weight[0])
    print(brain[0][1].weight)
    return brain

In [ ]:
def reproduce(brain_pair):
    brain_0 = brain_pair[0]
    brain_1 = brain_pair[1]
    for i, _ in enumerate(brain_0):
        if isinstance(brain_0[i], nn.Linear):
            # new_weights = layer.weight + ((torch.rand(layer.weight.shape)<self.mutation_rate).long() * torch.normal(layer.weight.shape) * self.mutation_change)
            random_assignment = torch.randint_like(brain_0[i].weight, 0, 2)
            new_weights = torch.mul(brain_0[i].weight, random_assignment) + torch.mul(brain_1[i].weight, 1-random_assignment)
            layer.weight = nn.Parameter(new_weights, requires_grad=False)
    return brain

In [78]:
z = [brains[x] for x in np.random.choice(top_25, 2)]
torch.randint_like(z[0])

TypeError: randint_like() received an invalid combination of arguments - got (Sequential), but expected one of:
 * (Tensor input, int high, *, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Tensor input, int low, int high, *, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
def evolve(top_25):
    new_brains = []
    for brain in top_25:
        new_brains.append(copy.deepcopy(brains[brain]))
        new_brains.append(mutate(brains[brain]))
    
    for _ in top_25:
        tuples = top_25
        new_brains.append(reproduce([brains[x] for x in np.random.choice(top_25, 2)]))
    brains = new_brains
    remaining = pop_size - len(brains)
    brains += [generate_individual_brain(input_size, num_hidden_layers, hidden_layers_size, output_size) for _ in range(remaining)]

In [42]:
def train_for_n_gen(self):
    # num_gen = self.num_gen
    for gen in range(num_gen):
        max_moves, max_scores, avg_scores = one_gen(board_size, set_seed, gen, window_size=self.input_window_size)
        # print(max_scores[np.argsort(max_scores)][::-1][:math.ceil(len(max_scores)/4)])
        # top_25 = np.argsort(max_scores)[::-1][:math.ceil(len(max_scores)/4)]
        top_25 = np.argsort(avg_scores)[::-1][:math.ceil(len(avg_scores)/4)]
        print(f"**END OF GEN {gen}**")
        print(f'max scores at the end of gen {gen} - {avg_scores}')
        print(f'max moves at the end of gen {gen} - {max_moves}')
        evolve(top_25)

In [72]:
pop_size = 50 # population size
num_trials = 20
move_limit = 100
num_gen = 50

mutation_rate = 0.15
mutation_change = 0.1
input_window_size = 3

input_size = (input_window_size * input_window_size) -1 + 6
num_hidden_layers = 4
hidden_layers_size = [25] *  num_hidden_layers
output_size = 4

brains = [generate_individual_brain(input_size, num_hidden_layers, hidden_layers_size, output_size) for _ in range(pop_size)]

In [44]:
board_size = 10
set_seed=True

In [45]:
max_moves, max_scores, avg_moves, avg_scores = one_gen(board_size, set_seed, 0, window_size=input_window_size)

In [56]:
final_scores = avg_scores + avg_moves/10
top_25 = np.argsort(final_scores)[::-1][:math.ceil(len(final_scores)/4)]

In [63]:
new_brains = []
for brain in top_25:
    new_brains.append(copy.deepcopy(brains[brain]))
    new_brains.append(copy.deepcopy(brains[brain]))
    new_brains.append(mutate(brains[brain]))
    new_brains.append(reproduce(brains[brain]))
brains = new_brains
remaining = pop_size - len(brains)
brains += [generate_individual_brain(input_size, num_hidden_layers, hidden_layers_size, output_size) for _ in range(remaining)]

NameError: name 'mutation_rate' is not defined

In [91]:
mutate(brains[brain])

TypeError: 'LayerNorm' object is not subscriptable

In [97]:

print(brains[brain][1].weight)

Parameter containing:
tensor([[-1.4797e-01, -1.9372e-01, -1.5614e-01, -1.6087e-01, -1.3738e-01,
          1.0097e-01,  2.5722e-01, -1.2398e-01, -2.3551e-01,  1.5597e-01,
         -2.6629e-03,  1.1633e-01,  2.0628e-01,  1.9024e-01],
        [-2.1496e-01, -4.9593e-02, -3.7448e-02, -2.2561e-02, -3.0399e-01,
          4.4416e-03, -8.6513e-02,  3.2931e-01, -1.8060e-01,  1.0163e-01,
         -2.5342e-01,  1.5035e-01, -2.1984e-01, -1.1093e-01],
        [ 9.1422e-02,  2.4928e-01, -2.4706e-01, -2.2503e-01, -1.1552e-01,
          2.2212e-01, -1.6447e-01, -2.1660e-01, -4.9307e-02,  7.8254e-02,
          8.7118e-04,  1.5589e-01, -2.2700e-01,  2.6106e-01],
        [ 1.8518e-01,  7.6916e-02, -3.1358e-02,  2.9195e-02,  2.1084e-01,
         -1.0000e-01,  1.0313e-01,  9.4177e-02,  4.6795e-02,  3.5747e-01,
         -1.3783e-01, -9.2377e-02, -1.4495e-01,  1.5124e-01],
        [ 1.0844e-01, -2.0042e-01,  8.1764e-02,  2.4534e-01,  8.5766e-02,
          1.7025e-01, -9.0083e-03,  1.3834e-01,  2.0383e-01, -6.

In [98]:
for layer in brains[brain]:
    if isinstance(layer, nn.Linear):
        # print('original')
        # print(layer.weight[0])
        # print('new')
        # new_weights = layer.weight + ((torch.rand(layer.weight.shape)<self.mutation_rate).long() * torch.normal(layer.weight.shape) * self.mutation_change)
        new_weights = layer.weight + ((torch.rand(layer.weight.shape)<mutation_rate).long() * torch.empty(layer.weight.shape).normal_(mean=0,std=1) * mutation_change)
        layer.weight = nn.Parameter(new_weights, requires_grad=False)
        # print(layer.weight[0])

In [107]:
z = [1,2,3]
for a in z:
    if a==3:
        a+=3
z

[1, 2, 3]

In [106]:
a

3